# Capstone proposal by Mirko Salomon 

# BEVERAGE MACHINE CHURN PREDICTION

* ## [1) Introduction and preparation](#Introduction) 

    *   
        #### Data Load
        
        #### Hyperparameters
        
* ## [2) XGBoost model](#XGB)
    
    *   
        #### Train the model
        
        #### Test data with best parameters 
        
        #### Confusion Matrix
        
        #### Tree Plot
        
        #### Prediction of churn

## 1) Introduction and preparation<a class="anchor" id="Introduction"></a>

Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function. Source : Wikipedia

For the accuracy score, gradient boosted trees generally perform better than Random forests and decision trees.

This model is a black box I will use it only if the results are better.

Can predict class probabilities and it seems to have good model performance (wins most of the Kaggle competitions)

I can easily fine tune and modify the model

In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np

import pandas as pd
import os
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import os
import pickle

# Specify the file path
file_path = r'C:\Users\msalomo\OneDrive - NESTLE\Certificate Machine Learning and Data\Churn Project\Notebook output'

import xlrd

import datetime as dt
from datetime import datetime

import collections
from collections import Counter

# Import seaborn
import seaborn as sns


In [2]:
#libraries
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import ParameterGrid

from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

### Data Load

In [3]:
# Specify the filename
filename = 'BM_noTickets_preprocess.p'

# Combine the file path and filename
file_path_with_filename = os.path.join(file_path, filename)

# Load the pickle file
with open(file_path_with_filename, 'rb') as file:
    BM_noTickets_preprocess = pickle.load(file)

In [4]:
# Load the pickle file 'X.p'
with open(os.path.join(file_path, 'X.p'), 'rb') as file:
    X = pickle.load(file)

# Load the pickle file 'y.p'
with open(os.path.join(file_path, 'y.p'), 'rb') as file:
    y = pickle.load(file)

# Load the pickle file 'X_tr.p'
with open(os.path.join(file_path, 'X_tr.p'), 'rb') as file:
    X_tr = pickle.load(file)

# Load the pickle file 'y_tr.p'
with open(os.path.join(file_path, 'y_tr.p'), 'rb') as file:
    y_tr = pickle.load(file)

# Load the pickle file 'X_val.p'
with open(os.path.join(file_path, 'X_val.p'), 'rb') as file:
    X_val = pickle.load(file)

# Load the pickle file 'y_val.p'
with open(os.path.join(file_path, 'y_val.p'), 'rb') as file:
    y_val = pickle.load(file)

# Load the pickle file 'X_te.p'
with open(os.path.join(file_path, 'X_te.p'), 'rb') as file:
    X_te = pickle.load(file)

# Load the pickle file 'y_te.p'
with open(os.path.join(file_path, 'y_te.p'), 'rb') as file:
    y_te = pickle.load(file)

In [5]:
BM_noTickets_preprocess.head()

,TA Contract Installation Date,Depreciation Start,TA Contract Start Date,TA Contract End Date,Churn,Service Category_Installation,Service Category_Removal,Service Category_Replacement,INCIDENT_CATEGORY_DESCRIPTION_Customer relocation,INCIDENT_CATEGORY_DESCRIPTION_Exchange / Replacement Sales,...,Generation_Gen. 2,Generation_Legacy,Blueprint Throughput_#-N/A,Blueprint Throughput_High,Blueprint Throughput_Low,Blueprint Throughput_Medium,IP Ownership_Exclusive,IP Ownership_Non-Proprietary,IP Ownership_Propr. Comp.,IP Ownership_Proprietary
0,1341.0,4079.0,1341.0,-485.000000,False,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1
1,1949.0,1067.0,1939.0,271.534348,False,2.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,1,0,0,0
2,1949.0,488.0,1939.0,271.534348,False,2.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,1,0,0,0
3,1949.0,1491.0,1939.0,271.534348,False,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,1
4,1949.0,3502.0,1939.0,271.534348,False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,1,0,0


### Hyperparameters

### class xgboost.XGBClassifier(**kwargs)
##### Bases: xgboost.sklearn.XGBModel, object

Implementation of the scikit-learn API for XGBoost classification.

##### Parameters

    n_estimators (int) – Number of boosting rounds.

    use_label_encoder (bool) – (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new code, we recommend that you set this parameter to False.

    max_depth (int) – Maximum tree depth for base learners.

    learning_rate (float) – Boosting learning rate (xgb’s “eta”)

    verbosity (int) – The degree of verbosity. Valid values are 0 (silent) - 3 (debug).

    objective (string or callable) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below).

    booster (string) – Specify which booster to use: gbtree, gblinear or dart.
    
"gbtree booster uses version of regression tree as a weak learner
gblinear uses (generalized) linear regression with l1&l2 shrinkage. But since it’s an additive process, and since linear regression is an additive model itself, only the combined linear model coefficients are retained.
dart adopted dropout method from neural networks to boosted regression rees. There are several parameters to be tuned:
•	skip_drop(default = 0, range [0, 1]) is the probability of skipping dropout. It has a higher priority than other DART parameters. If skip_drop = 1, the dropout procedure would be skipped and dart is the same as gbtree.
•	If skip_drop≠0, rate_drop (default = 0, range [0, 1]) will drop a fraction of the trees before the model update in every iteration. dropout makes dart between gbtree and random forest: “If no tree is dropped, dart is the same as (gbtree); if all the trees are dropped, dart is no different than random forest.”"
Source : 
https://xzz201920.medium.com/xgbosst-booster-gbtree-v-s-dart-v-s-gblinear-82d8fcbb07d2


    tree_method (string) – Specify which tree method to use. Default to auto. If this parameter is set to default, XGBoost will choose the most conservative option available. It’s recommended to study this option from parameters document.

    n_jobs (int) – Number of parallel threads used to run xgboost. When used with other Scikit-Learn algorithms like grid search, you may choose which algorithm to parallelize and balance the threads. Creating thread contention will significantly slow dowm both algorithms.

    gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.

    min_child_weight (float) – Minimum sum of instance weight(hessian) needed in a child.

    max_delta_step (int) – Maximum delta step we allow each tree’s weight estimation to be.

    subsample (float) – Subsample ratio of the training instance.

    colsample_bytree (float) – Subsample ratio of columns when constructing each tree.

    colsample_bylevel (float) – Subsample ratio of columns for each level.

    colsample_bynode (float) – Subsample ratio of columns for each split.

    reg_alpha (float (xgb's alpha)) – L1 regularization term on weights

    reg_lambda (float (xgb's lambda)) – L2 regularization term on weights

    scale_pos_weight (float) – Balancing of positive and negative weights.

    base_score – The initial prediction score of all instances, global bias.

    random_state (int) – Random number seed.


    missing (float, default np.nan) – Value in the data which needs to be present as a missing value.

    num_parallel_tree (int) – Used for boosting random forest.

    monotone_constraints (str) – Constraint of variable monotonicity. See tutorial for more information.

    interaction_constraints (str) – Constraints for interaction representing permitted interactions. The constraints must be specified in the form of a nest list, e.g. [[0, 1], [2, 3, 4]], where each inner list is a group of indices of features that are allowed to interact with each other. See tutorial for more information

    importance_type (string, default "gain") – The feature importance type for the feature_importances_ property: either “gain”, “weight”, “cover”, “total_gain” or “total_cover”.

Source :
Full documentation of parameters can be found here: https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst.

and https://xgboost.readthedocs.io/en/latest/python/python_api.html

## 2) XGBoost model<a class="anchor" id="XGB"></a>

### Train the model

pip install xgboost

### libraries
#from sklearn.tree import XGBClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Define a set of reasonable values
#n_estimators = range(10,200,50)

booster = ['gbtree', 'gblinear', 'dart']
eta = [0.01, 0.1, 0.3]

# Define a parameter grid of values
grid = ParameterGrid({
    'XGB__booster' : booster,
    'XGB__eta': eta
}
)
 
# Create pipeline, XGBoost classifier
pipe = Pipeline([
#('scaler', StandardScaler()),
('XGB', XGBClassifier(verbosity =0
    #scale_pos_weight=weight
))
])

   
# Save accuracy on test set
test_scores = []

for params_dict in grid:
    # Set parameters
    pipe.set_params(**params_dict)

    # Fit a k-NN classifier
    pipe.fit(X_tr, y_tr)

    # Save accuracy on validation set
    params_dict['accuracy'] = pipe.score(X_val, y_val)
    # Save f1 score on validation set
    # predict test instances
    y_pred = pipe.predict(X_val)
    params_dict['f1_macro'] = metrics.f1_score(y_val, y_pred, average='macro')
    
    # Save result
    test_scores.append(params_dict)
    
# Create DataFrame with test scores
scores_df = pd.DataFrame(test_scores)

# Top five scores
scores_df.sort_values(by='f1_macro', ascending=False).head(5)

I will try to improve the model with other parameters

# Define a set of reasonable values
#n_estimators = range(10,200,50)

booster = ['dart'] # 'gblinear' is removed, worst results with low score
eta = [0.1]  # 0.01
max_depth  = [6, 20, 50] #5, 6, 10
subsample = [0.5, 0.85] #0.25, 0.5, 0.9
min_child_weight =[1, 2] # 5, 10, 20

# Define a parameter grid of values
grid = ParameterGrid({
    'XGB__booster' : booster,
    'XGB__eta': eta,
    'XGB__max_depth': max_depth,
    'XGB__subsample': subsample,
    'XGB__min_child_weight': min_child_weight
}
)
 
# Create pipeline, XGBoost classifier
pipe = Pipeline([
#('scaler', StandardScaler()),
('XGB', XGBClassifier(verbosity =0
    #scale_pos_weight=weight
))
])

   
# Save accuracy on test set
test_scores = []

for params_dict in grid:
    # Set parameters
    pipe.set_params(**params_dict)

    # Fit a k-NN classifier
    pipe.fit(X_tr, y_tr)

    # Save accuracy on validation set
    params_dict['accuracy'] = pipe.score(X_val, y_val)
    # Save f1 score on validation set
    # predict test instances
    y_pred = pipe.predict(X_val)
    params_dict['f1_macro'] = metrics.f1_score(y_val, y_pred, average='macro')
    
    # Save result
    test_scores.append(params_dict)
    
# Create DataFrame with test scores
scores_df = pd.DataFrame(test_scores)

# Top five scores
scores_df.sort_values(by='f1_macro', ascending=False).head(5)

### Test data with best parameters

I test the model with the best parameters score from the validation data set

### TO transform into text - made to avoid preious long steps
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Define a set of reasonable values
#n_estimators = range(10,200,50)

 
# Create pipeline, XGBoost classifier
pipe = Pipeline([
#('scaler', StandardScaler()),
('XGB', XGBClassifier(verbosity =0
    #scale_pos_weight=weight
))
])

In [6]:
### libraries
#from sklearn.tree import XGBClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Define a parameter grid of values
params_dict = {
    'XGB__booster' : 'gbtree',
    'XGB__eta': 0.3,
    'XGB__max_depth': 30,
    'XGB__subsample': 0.75,
    'XGB__min_child_weight': 1
}

 
# Create pipeline, XGBoost classifier
pipe = Pipeline([
#('scaler', StandardScaler()),
('XGB', XGBClassifier(verbosity =0
    #scale_pos_weight=weight
))
])

# Save accuracy on test set
test_scores = []   

pipe.set_params(**params_dict)

# Fit a k-NN classifier
pipe.fit(X_tr, y_tr)

# Save accuracy on validation set
params_dict['accuracy'] = pipe.score(X_te, y_te)

# Save f1 score on test set
# predict test instances
y_pred = pipe.predict(X_te)

params_dict['f1_macro'] = metrics.f1_score(y_te, y_pred, average='macro')
    
# Save result
test_scores.append(params_dict)
    
# Create DataFrame with test scores
scores_df = pd.DataFrame(test_scores)

# Top five scores
scores_df

,XGB__booster,XGB__eta,XGB__max_depth,XGB__subsample,XGB__min_child_weight,accuracy,f1_macro
0,gbtree,0.3,30,0.75,1,0.894318,0.893244


The accuracy is 93% and F1 macro is 82%.

In [7]:
# F1_score
XGB_F1Macro = params_dict['f1_macro']

# Accuracy
XGB_accuracy = params_dict['accuracy']

In [8]:
# Load the npz file for results
# Specify the filename
filename = 'Results.npz'

# Combine the file path and filename
file_path_with_filename = os.path.join(file_path, filename)

# Load the npz file for results
with np.load(file_path_with_filename, allow_pickle=False) as npz_file:
    F1Score = npz_file['test_F1Score']
    accuracy = npz_file['test_accuracy']
    models = npz_file['models']
    
# Fill the calculated result value
F1Score[5] = XGB_F1Macro
accuracy[5] = XGB_accuracy

print('F1_Results:', F1Score)
print('Accuracy:', accuracy)

F1_Results: [0.42674315 0.         0.         0.         0.         0.89324379
 0.         0.        ]
Accuracy: [0.4895943  0.         0.         0.         0.         0.89431795
 0.         0.        ]


In [9]:
# Modify the Numpy array
#Model = models #unchanged
Result = F1Score
Accuracy = accuracy

# Store the changes in the results npz file
np.savez('Results.npz', models = models, test_F1Score = Result,  test_accuracy = Accuracy)

In [10]:
# Check the refreshed results
# Load the npz file for results
with np.load('Results.npz', allow_pickle=False) as npz_file:
    # It's a dictionary-like object    
    print(list(npz_file.keys()))
    # Load the arrays
    print('Models:', npz_file['models'])
    print('F1_Results:', npz_file['test_F1Score'])
    print('Accuracy:', npz_file['test_accuracy'])

['models', 'test_F1Score', 'test_accuracy']
Models: ['Baseline' 'Logistic Regression' 'KNeighbors' 'Decision Tree'
 'Random Forest' 'XGBoost' 'SelectedModel_wTickets'
 'SelectedModel_wTickets&Telemetry']
F1_Results: [0.42674315 0.         0.         0.         0.         0.89324379
 0.         0.        ]
Accuracy: [0.4895943  0.         0.         0.         0.         0.89431795
 0.         0.        ]


### Confusion Matrix

In [11]:
#Classification reports and confusion matrices are commonly used for reporting the performance of classifiers when working with imbalanced datasets.

# Print the confusion matrix
print(metrics.confusion_matrix(y_te, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_te, y_pred, digits=3))

[[26931  2241]
 [ 3482 21499]]
              precision    recall  f1-score   support

       False      0.886     0.923     0.904     29172
        True      0.906     0.861     0.883     24981

    accuracy                          0.894     54153
   macro avg      0.896     0.892     0.893     54153
weighted avg      0.895     0.894     0.894     54153



In [12]:
#Classification reports and confusion matrices are commonly used for reporting the performance of classifiers when working with imbalanced datasets.
ConfMat_df = pd.DataFrame(metrics.confusion_matrix(y_te, y_pred).T, columns=['False Condition', 'True Condition'],index=['Predicted False', 'Predicted True'])
ConfMat_df

,False Condition,True Condition
Predicted False,26931,3482
Predicted True,2241,21499


In [13]:
# Specify the filename
filename = 'XGBoost_ConfMat_df.p'

# Combine the file path and filename
file_path_with_filename = os.path.join(file_path, filename)

# Save the DataFrame into a pickle file
with open(file_path_with_filename, 'wb') as file:
    pickle.dump(ConfMat_df, file)

### Prediction of churn

# Need the one in the same format, so with removed Sales Org
# Load the pickle file
with open('BM_noTicketsWOSO.p', 'rb') as file:
    BM_noTicketsWOSO = pickle.load(file)

In [14]:
# Specify the filename
filename = 'BeverageMachine_withSerial.p'

# Combine the file path and filename
file_path_with_filename = os.path.join(file_path, filename)

# Load the pickle file
with open(file_path_with_filename, 'rb') as file:
    BeverageMachine_withSerial = pickle.load(file)

In [15]:
name = ['False', 'True']
No=BeverageMachine_withSerial['Serial ID']
predictions = pipe.predict_proba(X)
# With two column indices, values same  
# as dictionary keys 
df2 = pd.DataFrame(predictions, index=No ,columns = name) 
df2.head()

,False,True
Serial ID,,
132020210,0.998055,0.001945
19O0046537,0.850769,0.149231
23O0024982,0.975883,0.024117
HK10020559,0.898539,0.101461
4410837,0.978279,0.021721


In [16]:
df3 = df2.reset_index(level=None)
df3

,Serial ID,False,True
0,132020210,0.998055,0.001945
1,19O0046537,0.850769,0.149231
2,23O0024982,0.975883,0.024117
3,HK10020559,0.898539,0.101461
4,4410837,0.978279,0.021721
...,...,...,...
270759,7010049071,0.015675,0.984325
270760,7010042493,0.360982,0.639018
270761,7010059106,0.260657,0.739343
270762,7010053961,0.278136,0.721864


In [17]:
df3.reset_index(inplace=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270764 entries, 0 to 270763
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   index      270764 non-null  int64  
 1   Serial ID  270764 non-null  object 
 2   False      270764 non-null  float32
 3   True       270764 non-null  float32
dtypes: float32(2), int64(1), object(1)
memory usage: 6.2+ MB


df21 = pd.DataFrame(BeverageMachine_withSerial[['Serial ID','Churn']]).reset_index(level=None)
df21

In [18]:
df21 = pd.DataFrame(BeverageMachine_withSerial[['Serial ID','Churn', 'Sales Organisation']]).reset_index(level=None)
df21

,index,Serial ID,Churn,Sales Organisation
0,0,132020210,False,Nestlé Austria
1,1,19O0046537,False,Nestlé PH
2,2,23O0024982,False,Nestlé PH
3,3,HK10020559,False,Nestle Hong Kong
4,4,4410837,False,NP-Hungary
...,...,...,...,...
270759,270759,7010049071,True,Pakistan
270760,270760,7010042493,True,Pakistan
270761,270761,7010059106,True,Pakistan
270762,270762,7010053961,True,Pakistan


In [19]:
df21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270764 entries, 0 to 270763
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   index               270764 non-null  int64 
 1   Serial ID           270764 non-null  object
 2   Churn               270764 non-null  bool  
 3   Sales Organisation  270764 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 6.5+ MB


In [20]:
df3['Serial ID'] = df3['Serial ID'].astype('str')
df21['Serial ID'] = df21['Serial ID'].astype('str')
df3['index'] = df3['index'].astype('str')
df21['index'] = df21['index'].astype('str')


In [21]:
df3['KeyIndSer'] = df3['index'] + "-" + df3['Serial ID']
df21['KeyIndSer'] = df21['index'] + "-" + df21['Serial ID']

In [22]:
df4 = pd.merge(df3, df21, how='left', left_on = ['KeyIndSer'], right_on = ['KeyIndSer'])

In [23]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270764 entries, 0 to 270763
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index_x             270764 non-null  object 
 1   Serial ID_x         270764 non-null  object 
 2   False               270764 non-null  float32
 3   True                270764 non-null  float32
 4   KeyIndSer           270764 non-null  object 
 5   index_y             270764 non-null  object 
 6   Serial ID_y         270764 non-null  object 
 7   Churn               270764 non-null  bool   
 8   Sales Organisation  270764 non-null  object 
dtypes: bool(1), float32(2), object(6)
memory usage: 16.8+ MB


In [24]:
df4.to_csv(r'C:\Users\msalomo\predictions-Churn-XGBOOST.csv', index = False, header=True)


### Comments

Random forests generally outperform decision trees, but their accuracy is lower than gradient boosted trees. But in the end I might still use Decision Tree or Random Forest because the score is not based on the accuracy but on the F1_macro score. Here in this case the best model is Random Forest.